# Campus Recruitment

Campus placement is becoming highly competitive and there is immense load on colleges. This puts great pressures on the students if they are studying in some reputed college as the fear of not getting placed is constantly haunting them due to shallow fall in economy of country due to COVID-19. Here I will try to address the key dependencies of credentials earned from class 10th to current degree that would affect the chances of placement. Some key points undertaken are: -

1. Choice of board in class 10th and 12th to get placed.

2. Does gender effects the placements stats?

3. Work Experience, and internships effects.

4. What factors are responsible for not getting placed?

5. How does stream effects placement?

***

**At the end a model will be trained to perform predictive analysis**

***

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing important libraries and reading data

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline 

In [ ]:
df = pd.read_csv("../input/factors-affecting-campus-placement/Placement_Data_Full_Class.csv", engine = "python")
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

# Data Exploration

From info of the data, we can see the salary has only 148 entries and thus 67 entries are null value. This is due to the fact htat a guy is placed or not, thus for data cleansing ,we will remove the null values with some values helpful for us.(here zero)

In [ ]:
#Replacing all the null values with zero
df['salary'].fillna(0, inplace = True)

Now we will be replacing the string values with integer values for our understanding, such as status will be rplaced by 1 for placed and 0 for unplaced

In [ ]:
data = df

status = {'Placed': 1,'Not Placed': 0} 
data['status'] = [status[item] for item in data['status']] 

In [ ]:
df.describe()

In [ ]:
def myplot(data,x,y):
    plt.Figure(figsize =(12,12))
    sns.boxplot(x = data[x],y= data[y])
    g = sns.FacetGrid(data, row = y)
    g = g.map(plt.hist,x)
    plt.show()

In [ ]:
sns.set_style("ticks")
myplot(data,"salary","ssc_b")

Although the median salary for both central and state board remains same, the highest package given is higher
for students of central board. This happens because co-curricular activities provided to students are far
better than those of state board. Prominently the stage fear and hesitation cause is removed for students coming
from central board schools. From the histogram, it is seen that unplaced students are more for central schools.However overall placement stats remains same, thus choice of board for class 10th doesn't affects much for the placement.

In [ ]:
sns.set_style("ticks")
myplot(data, "salary", "gender")


<a href = "https://seaborn.pydata.org/generated/seaborn.FacetGrid.html" target="_blank"> reference</a> code for above plots

As from the boxplot we can see median salary for boys is greater than girls and so is the highest package. From the histogram, the number of unplaced girls and boys are 30 and 40 respectively. Althoug the number of unplaed girls seems less, but compared to the strngth of males and females, it is relatively high. 

In [ ]:
sns.countplot(data['status'],hue=data['gender'])

As we can see the number of placed students population of boys is higher than girls, although less number of girls did sit for the placement.This shows boys have slight better chance of getting hired than girls.

In [ ]:
fig, axes = plt.subplots(2,3, figsize=(12,12))
sns.barplot(x="hsc_s", y="status", data=data, ax = axes[(0,0)] )
sns.barplot(x="hsc_s", y="hsc_p", data=data, ax = axes[(0,1)])
sns.barplot(x="degree_t", y="status", data=data, ax = axes[(0,2)])
sns.barplot(x="status", y="degree_p", data=data, ax = axes[(1,0)])
sns.barplot(x="degree_t", y="degree_p", data=data, ax = axes[(1,1)])
plt.tight_layout(pad = 3)

From above plots it is clear that number of jobs available for commerce and science students is greater than that of students for other streams. Also it can be seen, the scores of class 12th do effect abit, although the impact is not that much high. Also the score of degree affects the placement, however maintaining an average score is enough to land in good jobs.

In [ ]:
fig, axes = plt.subplots(2,2, figsize=(12,12))
sns.barplot(x="workex", y="status", data=data, ax = axes[(0,0)])
sns.barplot(x="status", y="etest_p", data=data, ax = axes[(0,1)])
sns.barplot(x="specialisation", y="status", data=data, ax = axes[(1,0)])
sns.barplot(x="status", y="mba_p", data=data, ax = axes[(1,1)])
plt.tight_layout(pad = 3)

The first graph gives clear understanding that a work experience is preferred for getting into a good job,so one must go for good internships during college days. The employement test scores, doesn't matter much,and the scores of mba too, however a decent average score must be maintained. The marketing and finance student have more job opprtunities than H.R. for mba degree.

**Thus the key factor would be internship. Mostly the scores are discarded but they actually have some importance for the placement. Also chosing the stream puts huge impact for future growth of carrier**

# Data Preprocessing and Feature Engineering

1. Backward difference encoding of the categorical features and features containing texts.
2. Dropping the features not relevant or which puts less impact on the model.
3. Extracting important features.
4. Using PCA/t-SNE for distribution visualization.
5. Standardizing the dataset before training.

## Encoding the data


In [ ]:
# encoding for the features

import category_encoders as ce
encoder = ce.BackwardDifferenceEncoder(cols=['ssc_b', "hsc_b", "hsc_s", "degree_t", "workex", "specialisation", "gender"])
data_new = encoder.fit_transform(data)

data_new.head()


The code for backward difference encoding is taken from the mentioned <a href="https://www.datacamp.com/community/tutorials/categorical-data" target="_blank">source </a> .
In dataset as shown above we have encoded the string categorical data with some encoded values, so that it becomes
relevant for models to learn. One hot encoding may suffer with the problem of curse of dimensionality, thus backward difference encoding is used.

## Dropping unneccessary features


In [ ]:
data_new.drop(['intercept','sl_no'], axis=1, inplace=True)
data_new.head()

## Extracting important features


In [ ]:
x = data_new["salary"]
labels = data_new['status']
features = data_new.iloc[:, :-2 ]
features = pd.concat([features, x], axis=1, join='inner')
features.head()


In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# feature extraction
model = LogisticRegression(solver='lbfgs')
rfe = RFE(model, 10)
fit = rfe.fit(features, labels)
print("Num Features: %d" % fit.n_features_)
print("Selected Features: %s" % fit.support_)
print("Feature Ranking: %s" % fit.ranking_)

We will use these features for training our ML algorithm and further look over the results. I chose 10 features important for the training purpose, although one is free to chose of its own.The important features for placement stats are: **["ssc_p", "hsc_p", "hsc_s_0", "hsc_s_1", "degree_p", "degree_t_1", "workex_0", "etest_p", "mba_p", "salary"]** To have some insights about features we will use dimensionality reductin in below setion and have look how much separability in the features is present.
The code snippet for feature selection is taken from https://machinelearningmastery.com/feature-selection-machine-learning-python/.

Some other sources from where help is taken for feature selection is https://www.datacamp.com/community/tutorials/feature-selection-python

We will keep only the 10 most important features for working further.

In [ ]:
#dropping the features
features.drop(["gender_0", "ssc_b_0", "hsc_b_0", "degree_t_0", "specialisation_0"], axis=1, inplace=True)
features.head()

## Applying t-SNE and using it for getting insight of features


In [ ]:
df_final = features
df_final.to_numpy()
distinct_labels = list(set(labels))

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.patheffects as PathEffects

y = labels
X_raw = df_final
y_raw = np.array(y, dtype = 'int')
tsne = TSNE(n_components=2, random_state=0, perplexity = 50, n_iter = 5000)
X_2d = tsne.fit_transform(X_raw)
X1 = X_2d[:,0:1]
Y1 = X_2d[:,1:2]

sns.set_style('ticks')
sns.set_palette('muted')
sns.set_context("notebook", font_scale=1.5,
                rc={"lines.linewidth": 2.5})


category_to_color = {0: 'red', 1: 'blue'}
category_to_label = {0: 'Unplaced', 1:"Placed"}

fig, ax = plt.subplots(1,1)
for category, color in category_to_color.items():
    mask = y == category
    ax.plot(X_2d[mask, 0], X_2d[mask, 1], 'o',
            color=color, label=category_to_label[category], ms = 6)

ax.legend(loc='best')
ax.axis('on')
ax.axis('tight')
plt.xlabel('Dimension1')
plt.ylabel('Dimension2')
plt.title(' t-SNE plot')

Looking at the above t-SNE plot it is very easy to conclude that features are separable, also the two classes are shown. Now we will move further to train models and do predictive analysis and verify models by using different metrices for our classifiers or regressor models. Before this there is one last step of normalizing the dataset so that all the values lies in a particular range for the robustness of our model.

## Standardizing the data

In [ ]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
x = scaler.fit_transform(X_raw)
y = y_raw

# Training different classifiers and measuring the accuracy values


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.18)

In [ ]:
model1 = RandomForestClassifier()
model1.fit(x_train,y_train)
model1.score(x_test,y_test)
predictions1 = model1.predict(x_test)
print(confusion_matrix(y_test,predictions1))
print(classification_report(y_test,predictions1))

In [ ]:
model2 = DecisionTreeClassifier()
model2.fit(x_train,y_train)

predictions2 = model2.predict(x_test)
print(confusion_matrix(y_test,predictions2))
print(classification_report(y_test,predictions2))

In [ ]:
model3 = KNeighborsClassifier()
model3.fit(x_train,y_train)

predictions3 = model3.predict(x_test)
print(confusion_matrix(y_test,predictions3))
print(classification_report(y_test,predictions3))

In [ ]:
model4 = XGBClassifier()
model4.fit(x_train,y_train)
predictions4 = model4.predict(x_test)
print(confusion_matrix(y_test,predictions4))
print(classification_report(y_test,predictions4))

In [ ]:
model5 = GaussianNB()
model5.fit(x_train,y_train)
predictions5 = model5.predict(x_test)
print(confusion_matrix(y_test,predictions5))
print(classification_report(y_test,predictions5))

In [ ]:
#neural network classifier
model6 = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)
model6.fit(x_train,y_train)
predictions6 = model6.predict(x_test)
print(confusion_matrix(y_test,predictions6))
print(classification_report(y_test,predictions6))

Looking at the confusion matrix, it appears that all the models are performing exceptionally well. Some reasons which I can think of is: - 
1. Since addressing the case of placed and unplaced becomes binary classification problem, so most of the models are able to perform well as the dataset size is too small.
2. The t-SNE plot shows that both the classes are ar too apart and thus the features are searable, due to which complexity has almost gone down.
3. Feature selection step extracts only the most important features, due to which the data becomes quite clean for training purpose.
4. Also the presence of outlier is not there or is minimal as we can see from the plots above.

# Conclusion and Discussion
1. We can say that for classification model, the results are pretty well and if the student has some decent credentials in college and better 12th score, there might be chances of earning good salary job.
2. The gender is taken into factor for placement, however it is very less.
3. Co- curriculars do affects as it adds to the holistic personality of the student.
4. Most important point is work experience and internships do effects the placements and obviously one must go for it during college days.
5. Another key factor is stream choice as science and commerce have better jobs than arts, however only chosing a stream won't help to earn decent job,rather one must take into account above mentioned factors.